4. Метрики в рекомендательных системах

In [35]:
from sklearn.metrics import ndcg_score, dcg_score
import numpy as np
import math

true = np.asarray([[3, 2, 1, 0, 0]])
relevance = np.asarray([[3, 2, 0, 0, 1]])

print(ndcg_score(true, relevance))

#0.980840401274087

0.980840401274087


Задание 4.5
1 point possible (graded)
Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

Реальные оценки: [2, 4, 1, 1, 1]
Предсказанные оценки: [2, 5, 2, 3, 1]
Вычислите коэффициент NDCG. Округлите результат до двух знаков после точки-разделителя.

In [3]:
true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])

print(ndcg_score(true, relevance))


0.9748317848747132


5. Проблема холодного старта и popularity-based model

In [4]:
import pandas as pd

In [5]:
movie = pd.read_csv('data/movie.zip')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
rating = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
df=pd.merge(rating,movie, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Задание 5.1
1 point possible (graded)
Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

In [8]:
df['title'].value_counts()[:5]

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

Задание 5.2
1 point possible (graded)
Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных Movie Lens есть оценки каждого из зрителей.

Найдите средний рейтинг для каждого из фильмов.
Найдите фильмы с наивысшим средним рейтингом.
Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.
Впишите только название фильма, без кавычек и без года выхода на экран.

Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

  нет ответа 

In [9]:
rate = df.groupby('title').mean('rating')

In [10]:
rate[rate['rating']==5].sort_index().tail(1)

,userId,movieId,rating
title,,,
Yonkers Joe (2008),54248.0,119430.0,5.0


Задание 5.3
1 point possible (graded)
В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

Сколько таких фильмов?

In [11]:
movie_list = df['title'].value_counts().gt(50)
movie_list = movie_list[movie_list==True].index
len(movie_list)

10472

Задание 5.4
1 point possible (graded)
Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей средней оценкой. В качестве ответа запишите название этого фильма без артикля и года выхода на экран.
Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

In [12]:
df[df['title'].isin(movie_list)].groupby('title').mean('rating').sort_values('rating', ascending=False).head(1)

,userId,movieId,rating
title,,,
"Shawshank Redemption, The (1994)",69205.346164,318.0,4.44699


6. Практика

Задание 6.1
1 point possible (graded)
Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [16]:
articles = pd.read_csv('data/shared_articles.zip')
articles =  articles[articles['eventType'] == 'CONTENT SHARED']
articles.shape[0]

3047

In [18]:
users = pd.read_csv('data/users_interactions.zip')
users.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [19]:
users.personId = users.personId.astype(str)
users.contentId = users.contentId.astype(str)
articles.contentId = articles.contentId.astype(str)

Задание 6.2
1 point possible (graded)
Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [20]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [21]:
users['score'] = users['eventType'].apply(lambda x: event_type[x])

In [23]:
round(users['score'].mean(), 2)

1.24

Задание 6.3
1 point possible (graded)
Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [32]:
iter_count = users.groupby('personId').nunique('contentId')
iter_count = iter_count[iter_count['contentId'] >= 5]
len(iter_count.index)

1140

Задание 6.4
1 point possible (graded)
Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

В этом задании необходимо работать с данными, полученными в задании 6.3.

In [34]:
users = users[users['personId'].isin(iter_count.index)]

users.shape[0]

69868

Задание 6.5
1 point possible (graded)
Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

В этом задании необходимо работать с данными, полученными в задании 6.4.

In [36]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [49]:
users_score = users.groupby(['personId', 'contentId'], as_index=False).sum('score')
timestamp = users.groupby(['personId', 'contentId'], as_index=False).max('timestamp')

users_score['score'] = users_score['score'].apply(smooth_user_preference)

users_grouped = pd.merge(users_score, timestamp, how='inner',on=['personId', 'contentId'])

round(users_grouped.timestamp_y.mean(), 2)

1470605340.04

Задание 6.6
1 point possible (graded)
Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [50]:
users_train = users_grouped[users_grouped.timestamp_y < 1475519545]
users_test = users_grouped[users_grouped.timestamp_y >= 1475519545]

users_train.shape[0]

29325

Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [51]:
final_df = (
    users_train.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    users_test.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


Задание 6.7
1 point possible (graded)

Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [59]:
recomend_list = users_train.groupby('contentId').sum('score_x').sort_values('score_x', ascending=False).index
recomend_list[0]

'-6783772548752091658'

Задание 6.8
1 point possible (graded)
Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления precision@10 воспользуйтесь следующей функцией:

def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
Итоговый результат округлите до трёх знаков после точки-разделителя.

В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).

In [73]:
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()

In [70]:
def recomend(x):
    recomendation=[]
    for i in recomend_list:
        if i not in x:
            recomendation.append(i)
        if len(recomendation)>=10:
            break
    return recomendation
    

In [71]:
final_df['offer'] = final_df['true_train'].apply(recomend)
final_df

,true_train,true_test,offer
personId,,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72...","[-6783772548752091658, -133139342397538859, -8..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -...","[-6783772548752091658, -133139342397538859, -8..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -...","[-6783772548752091658, -8208801367848627943, 8..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -...","[-133139342397538859, -8208801367848627943, 82..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224],"[-6783772548752091658, -8208801367848627943, 8..."
...,...,...,...
953707509720613429,"[-1068603220639552685, -2358756719610361882, -...","[-2402288292108892893, -5813211845057621660, -...","[-133139342397538859, -8208801367848627943, 82..."
983095443598229476,"[-133139342397538859, -8742648016180281673]","[-14569272361926584, -1572252285162838958, -18...","[-6783772548752091658, -8208801367848627943, 8..."
989049974880576288,"[-133139342397538859, -2038869595290705317, -2...","[-6289909056857931861, -7047448754687279385, -...","[-6783772548752091658, -8208801367848627943, 8..."


In [74]:
precision('offer')

0.006454207722621089